# Plotting traces (collections)
This Tutorial explains how to customize network plots in pandapower using plotly. Each pandapower network element can be translated into a plotly trace with all corresponding properties...

In [ ]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.plotting.plotly as pplotly
from pandas import Series

If you want to have full control over the layout of your plot, you can individually create and plot collections with the pandapower plotting module.

In [ ]:
net = nw.mv_oberrhein()

lc = pplotly.create_line_trace(net,net.line.index, color='black')
bc = pplotly.create_bus_trace(net, net.bus.index, size=10, color="orange",
                              infofunc=Series(index=net.bus.index,
                                              data=net.bus.name + '<br>' + net.bus.vn_kv.astype(str) + ' kV'))

pplotly.draw_traces(bc + lc, figsize=1, aspectratio=(8,6))

Order of ploting traces is as ordered in the `draw_traces` function. So, in order to plot buses in front of lines first lines and then buses need to be set in the input traces list:

In [ ]:
pplotly.draw_traces(lc + bc , figsize=1, aspectratio=(8,6))

## Highligting
Specific lines or buses can be highlighted by creating extra line collections in different colors.

In this example, we plot lines that are longer than 2 km green and buses with a voltage below 0.98 pu red.

First, we create a line collection for all lines in grey and a line collection for only the long lines in green:

In [ ]:
long_lines = net.line[net.line.length_km > 2.].index
lc = pplotly.create_line_trace(net, net.line.index, color="grey")
lcl = pplotly.create_line_trace(net, long_lines, color="green", width=2,
                                infofunc=Series(index=net.line.index, 
                                                data=net.line.name[long_lines] + '<br>' + net.line.length_km[long_lines].astype(str) + ' km'))

low_voltage_buses = net.res_bus[net.res_bus.vm_pu < 0.98].index
bc = pplotly.create_bus_trace(net, net.bus.index, size=10, color="blue")
bch = pplotly.create_bus_trace(net, low_voltage_buses, size=10, color="red")

pplotly.draw_traces(bc + bch + lc + lcl )

## Highlighting with the Topology Package
Colors palette for plotly can be obtained using function `plotting.plotly.get_plotly_color_palette(n)` where argument `n` defines number of colors that will be returned in a list.

In [ ]:
from pandapower.plotting.plotly import get_plotly_color_palette

net = nw.mv_oberrhein()

mg = pp.topology.create_nxgraph(net, nogobuses=set(net.trafo.lv_bus.values) | set(net.trafo.hv_bus.values))
collections = []
ai = 0
islands = list(pp.topology.connected_components(mg)) # getting connected components of a graph
colors = get_plotly_color_palette(len(islands)) # getting a color for each connected component
for color, area in zip(colors, islands):
    collections += pplotly.create_bus_trace(net, area, size=5, color=color, trace_name='feeder {0}'.format(ai))
    ai += 1
collections += pplotly.create_line_trace(net, net.line.index, color="grey")
collections += pplotly.create_bus_trace(net, net.ext_grid.bus.values, patch_type="square", size=10,
                                        color="yellow")
pplotly.draw_traces(collections)

## Collection on mapbox
If network geo-data are not in lat/long form, there is a function `geo_data_to_latlong` which can be used to transform entire network geodata from a specific projection to lat/long:

In [ ]:
net = nw.mv_oberrhein()
pplotly.geo_data_to_latlong(net, projection='epsg:31467') #transforming geodata to lat/long

lc = pplotly.create_line_trace(net, net.line.index, color='purple')
bc = pplotly.create_bus_trace(net, net.bus.index,size=10,color="orange",
                              infofunc=Series(index=net.bus.index,
                                              data=net.bus.name + '<br>' + net.bus.vn_kv.astype(str) + ' kV'))
pplotly.draw_traces(lc + bc, on_map=True, map_style='dark')

## Colormaps
Colormaps plots are available for plotly by setting `cmap=True` in create functions. By default line loading and bus voltage magnitudes will be used for coloring.

In [ ]:
net = nw.mv_oberrhein()
bt = pplotly.create_bus_trace(net, cmap=True)
lt = pplotly.create_line_trace(net, cmap=True)
pplotly.draw_traces(lt + bt, showlegend=False)

Some customization is possible in regard to: limits, choosing a colormap, colorbar title, etc.

In [ ]:
lt = pplotly.create_line_trace(net, cmap='Blues', cmin=0, cmax=100, cbar_title = 'Line loading [%]')
pplotly.draw_traces(lt, showlegend=False)

Alternatively, any other values can be used for colormaps, with also some customization in regard to limits, choosing a colormap, colorbar title, etc. An axample with coloring according to line length:

In [ ]:
lt = pplotly.create_line_trace(net, cmap_vals=net.line.length_km, cmap=True, cbar_title = 'Line length [km]')
pplotly.draw_traces(lt, showlegend=False)

## Customized hover-info
Information that pop-ups when a bus or a line is pointed with mouse-cursor is called hover-info. By default it is set to bus/line name, but it can be customized by setting a list of strings to `infofunc` argument. A newline is defined using `<br>`. See a representative example hereafter (try getting hoverinfo by pointing a cursor above each bus):

In [ ]:
bc = pplotly.create_bus_trace(net, [1,2,3,4,5], size=15,
                              infofunc=Series(index=net.bus.index, data=net.bus.name + '<br>' + 
                                              net.bus.vn_kv.astype(str) + ' kV' + '<br>' +
                                              'this is one of the 5 buses...'))
pplotly.draw_traces(bc)

More tutorials about interactive plots using ploltly:
* [built-in interactive plots](http://nbviewer.jupyter.org/github/e2nIEE/pandapower/blob/develop/tutorials/plotly_built-in.ipynb)
* [interactive plots on maps](http://nbviewer.jupyter.org/github/e2nIEE/pandapower/blob/develop/tutorials/plotly_maps.ipynb)